In [1]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 14 not upgraded.
Need to get 81.0 MB of archives.
After this operation, 273 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 87.0.4280.66-0ubuntu0.18.04.1 [1,122 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 87.0.4280.66-0ubuntu0.18.04.1 [71.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 87.0.428

In [4]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [12]:
url = 'https://google.com'
wd = webdriver.Chrome('chromedriver',options=options)
wd.get(url)

In [6]:
cd /content/drive/MyDrive/Colab Notebooks/Real_Estate/S_Real_Estate

/content/drive/MyDrive/Colab Notebooks/Real_Estate/S_Real_Estate


In [7]:
ls

chromedriver                     links_list.txt
crawl_data_batdongsan.com.vn.py  Selenium_Real_Estate
data_batdongsan_com_vn.csv


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import json
import csv

fields = ['Tên', 'Mức giá', 'Diện tích', 'Loại tin đăng', 'Địa chỉ',
          'Mặt tiền', 'Đường vào', 'Hướng ban công', 'Số tầng', 'Số phòng ngủ',
          'Số toilet', 'Nội thất', 'Pháp lý', 'Tên dự án', 'Chủ đầu tư',
          'Quy mô', 'Ngày đăng', 'Ngày hết hạn', 'Loại tin', 'Mã tin', 'Phòng ngủ', 'Hướng nhà']

def write_csv(dictionary):
    file_path = "data_batdongsan_com_vn.csv"
    try:
        with open(file_path, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fields)
            #writer.writeheader()
            #for data in dictionary:
            writer.writerow(dictionary)
    except IOError:
        print("I/O error")

def extract_title(driver):
    title= driver.find_elements_by_xpath('//div[@class="description"]/h1[@class="tile-product"]')[0].text
    dic = dict()
    dic['Tên'] = title
    #print(dic)
    return dic

def extract_price_and_area(driver):
    short_detail_get = driver.find_elements_by_xpath('//ul[@class="short-detail-2 clearfix pad-16"]/li/span')
    short_detail_list = []
    for i in range(len(short_detail_get)):
        short_detail_list.append(short_detail_get[i].text)

    dic = dict()
    key = ''
    value = 0
    bound = len(short_detail_list)
    i = 0
    while i < bound:
        if i % 2 == 0:
            key = short_detail_list[i][:-1]
        else:
            value = short_detail_list[i]
            dic[key] = value

        i = i + 1

    #print(dic)
    return dic

def extract_detail(driver):
    detail_product_get = driver.find_elements_by_xpath(
        '//div[@class="box-round-grey3"]/div/span')
    detail_product_list = []
    for i in range(len(detail_product_get)):
        detail_product_list.append(detail_product_get[i].text)

    dic = dict()
    key = ''
    value = 0
    bound = len(detail_product_list)
    i = 0
    while i < bound:
        if i % 2 == 0:
            key = detail_product_list[i][:-1]
        else:
            value = detail_product_list[i]
            dic[key] = value

        i = i + 1

    #print(dic)
    return dic

def extract_date_and_id(driver):
    date_and_id_get = driver.find_elements_by_xpath('//ul[@class="short-detail-2 list2 clearfix"]/li/span')
    date_and_id = []
    for i in range(len(date_and_id_get)):
        date_and_id.append(date_and_id_get[i].text)

    dic = dict()
    key = ''
    value = 0
    bound = len(date_and_id)
    i = 0
    while i < bound:
        if i % 2 == 0:
            key = date_and_id[i][:-1]
        else:
            value = date_and_id[i]
            dic[key] = value

        i = i + 1

    #print(dic)
    return dic

if __name__ == '__main__':
    
    driver = webdriver.Chrome('chromedriver',options=options)
    file_path = 'links_list.txt'
    with open(file_path) as f:
        links_list = f.readlines()

    order = 10000
    end = 20000

    for i in range(order, end):
        link = links_list[i]
        print('-----------------------------------------------------------')
        print(order, '- ', link)
        #link_test = 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-minh-khai-phuong-vinh-tuy-prj-imperia-sky-garden/-chinh-chu-ban-nhanh-goc-dong-nam-toa-c-full-100-noi-that-view-dep-gia-4-5x-ty-pr27951316'
        driver.get(link)
        data = dict()


        try:
            #Get title:
            title = extract_title(driver)

            #Get price and area
            price_area = extract_price_and_area(driver)

            #Get detail:
            detail = extract_detail(driver)

            #Get date and id
            date_and_id = extract_date_and_id(driver)

            #involving
            data.update(title)
            data.update(price_area)
            data.update(detail)
            data.update(date_and_id)

            print(data)

            write_csv(data)
            order = order + 1
            with open('start_end.txt', 'a') as f:
                f.writelines(str(order) + '\n')

            # stop = stop + 1
            # if (stop == 2):
            #     break

        except:
            print('Item is not available or Error')


#434 -  https://batdongsan.com.vn/ban-can-ho-chung-cu-pho-phung-hung-1-phuong-phuc-la-prj-khu-do-thi-moi-xa-la/sieu-pham-nha-dep-gia-tot-tai-ct1a-90m-bc-dong-nam-full-noi-that-gia-1-52-ty-tl-manh-pr28084191
#502


-----------------------------------------------------------
10000 -  https://batdongsan.com.vn/ban-can-ho-chung-cu-pho-cau-am-phuong-van-phuc-1-prj-goldsilk-complex/chinh-chu-khong-qua-mg-ban-77m2-du-an-goldsik-2-ngu-bc-dong-nam-toa-j-tang-24-pr26717214

{'Tên': 'Chính chủ không qua MG bán căn 77m2 dự án Goldsilk Vạn Phúc, 2PN BC Đông Nam, tòa J tầng 24', 'Mức giá': '2.2 tỷ', 'Diện tích': '77 m²', 'Phòng ngủ': '2 PN', 'Loại tin đăng': 'Bán căn hộ chung cư', 'Địa chỉ': 'Dự án GoldSilk Complex, Phố Cầu Am, Phường Vạn Phúc, Hà Đông, Hà Nội', 'Hướng nhà': 'Tây-Bắc', 'Hướng ban công': 'Đông-Nam', 'Số phòng ngủ': '2 (phòng)', 'Số toilet': '2 (phòng)', 'Nội thất': 'Full nội thất.', 'Pháp lý': 'Đã có sổ hồng.', 'Tên dự án': 'Goldsilk Complex· Xem dự án', 'Chủ đầu tư': 'Công ty CP Bất động sản Hanovid', 'Quy mô': 'Gồm 01 tổ hợp thương mại và nhà ở gồm 2 tòa nhà tháp cao 32 tầng', 'Ngày đăng': '04/12/2020', 'Ngày hết hạn': '11/12/2020', 'Loại tin': 'Tin thường', 'Mã tin': '26717214'}
Item is not